In [2]:
import logging, json, os, requests, datetime
import azure.functions as func
from azure.servicebus import ServiceBusClient, ServiceBusMessage


In [3]:
output = {
    "key": "", 
    "file_path": "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/rest-api/layout.png", 
    "model": "prebuilt-layout", 
    "operation_location": "", 
    "Operation-location": "https://formr-pdf.cognitiveservices.azure.com/formrecognizer/documentModels/prebuilt-layout/analyzeResults/4dbb0d6a-d743-4d91-9e83-61f26efc3a12?api-version=2022-08-31"}

In [4]:
msg = json.dumps(output, indent=8) # dict to string
print(msg)
data = json.loads(msg)

{
        "key": "",
        "file_path": "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/rest-api/layout.png",
        "model": "prebuilt-layout",
        "operation_location": "",
        "Operation-location": "https://formr-pdf.cognitiveservices.azure.com/formrecognizer/documentModels/prebuilt-layout/analyzeResults/4dbb0d6a-d743-4d91-9e83-61f26efc3a12?api-version=2022-08-31"
}


In [5]:
cog_endpoint = "https://formr-pdf.cognitiveservices.azure.com/"
cog_key = "c034489b5cc34af187712e6db5a2b756"
model= "prebuilt-layout"
form_recognizer_url = f"{cog_endpoint}formrecognizer/documentModels/{model}:analyze?api-version=2022-08-31"


In [6]:
r = requests.get(
    data['Operation-location'], 
    headers={'Ocp-Apim-Subscription-Key': cog_key}
    )

In [7]:
status=r.json()['status']
print(status)

if status == 'succeeded':
        data['results'] = r.json()
elif status in ('notStarted', 'running'):
        pass

succeeded


In [9]:
res=data['results']['analyzeResult']['tables']
print("found" , len(res), 'tables')

found 2 tables


In [10]:
#import pandas as pd

tables=[]
if 'tables' in data['results']['analyzeResult'].keys():
    res=data['results']['analyzeResult']['tables']
    for table in res:
        raw_text=[]
        cells = []
        headers=[]
        regions=table['boundingRegions']
        for r in regions: pageNumber = r['pageNumber']
        for cell in table['cells']:
            cells.append(
            {
                "text": cell['content'],
                "rowIndex": cell['rowIndex'],
                "colIndex": cell['columnIndex'],
                "confidence": '',
                "is_header": False
            })
            isheader=False
            if ('kind' in cell.keys() ): 
                isheader = (cell['kind']=='columnHeader')
                cells.append({'is_header': isheader})
                if isheader: 
                    headers.append(cell['content'])
            if not isheader: 
                raw_text.append(cell['content'])


        tables.append({
            "page_number": pageNumber,
            "raw_count" : table['rowCount'],
            "column_count": table['columnCount'],
            "headers": headers,
            "cells": cells,
            "raw_text":raw_text
        }
        )
        h=len(headers)
        #df = pd.DataFrame([raw_text[i:i+h] for i in range(0, len(raw_text), h)],columns=headers)
        


In [12]:
import pandas as pd

t= tables[0]
r=t['raw_text']
h=t['headers']
df = pd.DataFrame([r[i:i+len(h)] for i in range(0, len(r), len(h))],columns=t['headers'])
df

,From,To,By,ETD,ETA
0,"118 Queen Street Hoboken, NJ 07030","52 West Trenton St. Harleysville, PA 19438",cause science slow,09-Dec-2018 19:00,09-Dec-2020 11:00
1,"9 Ketch Harbour Ave. Vincentown, NJ","75 Fawn Street Peabody, MA 01960",tone late spoken,12-Dec-2018 10:00,19-Dec-2020
